# Claude CLI - Agents et Subagents

**Module :** Vibe-Coding / Claude Code / Notebooks CLI  
**Niveau :** Intermediaire  
**Duree :** 30 min  
**Prerequis :** Notebooks 01-03 completes

## Objectifs d'Apprentissage

- [ ] Comprendre les agents integres (Explore, Plan)
- [ ] Utiliser l'agent Explore pour rechercher du code
- [ ] Utiliser l'agent Plan pour planifier des taches
- [ ] Creer des subagents personnalises
- [ ] Paralleliser des taches avec plusieurs agents

---

## 1. Configuration

In [ ]:
import sys
import subprocess
import json
import os

sys.path.insert(0, 'helpers')
from claude_cli import run_claude, verify_installation, print_response

EXAMPLES_DIR = os.path.join(os.getcwd(), 'examples', 'sample_project')
print(f"Claude CLI pret: {verify_installation()}")

## 2. Concept d'Agents

Claude Code dispose d'**agents specialises** qui peuvent effectuer des taches complexes de maniere autonome :

| Agent | Fonction | Outils disponibles |
|-------|----------|-------------------|
| **Explore** | Recherche rapide en lecture seule | Read, Glob, Grep |
| **Plan** | Planification et architecture | Read, Glob, Grep + analyse |
| **General** | Taches generales | Tous les outils |

```
┌─────────────────────────────────────┐
│          Claude Principal           │
│  (Coordonne les taches complexes)   │
└───────────────┬─────────────────────┘
                │
    ┌───────────┼───────────┐
    ▼           ▼           ▼
┌───────┐  ┌───────┐  ┌───────┐
│Explore│  │ Plan  │  │General│
│ Agent │  │ Agent │  │ Agent │
└───────┘  └───────┘  └───────┘
```

## 3. Agent Explore

L'agent **Explore** est optimise pour la recherche rapide dans un codebase :

- **Lecture seule** : Ne modifie rien
- **Rapide** : Utilise un modele leger
- **Outils** : Glob, Grep, Read

En mode interactif, Claude lance automatiquement des agents Explore. En CLI, on peut simuler ce comportement.

In [ ]:
# Simuler une recherche de type Explore
# En pratique, Claude utilise l'agent Explore automatiquement

stdout, stderr, code = run_claude(
    f"""Explore ce projet et reponds a ces questions:
1. Quelles sont les principales fonctions ?
2. Y a-t-il des tests ?
3. Quelles dependances externes sont utilisees ?

Projet:
- main.py : point d'entree
- utils.py : fonctions utilitaires
- tests/test_utils.py : tests unitaires""",
    model="haiku"  # Modele rapide comme un agent Explore
)
print_response(stdout, stderr, code)

In [ ]:
# Recherche de patterns specifiques (comme Grep)
with open(os.path.join(EXAMPLES_DIR, 'utils.py'), 'r') as f:
    utils_content = f.read()

stdout, stderr, code = run_claude(
    f"""Dans ce code, trouve toutes les fonctions qui:
1. Prennent une liste en parametre
2. Retournent un dictionnaire

Code:
{utils_content}""",
    model="haiku"
)
print_response(stdout, stderr, code)

## 4. Agent Plan

L'agent **Plan** aide a structurer des taches complexes :

- Analyse l'existant
- Propose une architecture
- Decompose en etapes
- Identifie les risques

In [ ]:
# Demander un plan d'implementation
stdout, stderr, code = run_claude(
    """Je veux ajouter les fonctionnalites suivantes au projet sample_project:

1. Export des resultats en CSV
2. Graphiques avec matplotlib
3. Interface en ligne de commande (CLI)

Cree un plan d'implementation detaille avec:
- Les fichiers a creer/modifier
- L'ordre des etapes
- Les dependances a ajouter
- Les risques potentiels"""
)
print_response(stdout, stderr, code)

In [ ]:
# Plan de refactoring
stdout, stderr, code = run_claude(
    f"""Propose un plan de refactoring pour ce code:

{utils_content}

Objectifs:
- Ameliorer la lisibilite
- Ajouter la gestion d'erreurs
- Permettre l'extension future

Format: etapes numerotees avec justification"""
)
print_response(stdout, stderr, code)

## 5. Subagents Personnalises

Vous pouvez definir vos propres agents specialises avec un fichier JSON :

```json
{
  "reviewer": {
    "description": "Expert en revue de code",
    "prompt": "Tu es un senior code reviewer...",
    "tools": ["Read", "Grep"],
    "model": "sonnet"
  }
}
```

In [ ]:
# Definition d'agents personnalises
custom_agents = {
    "security_reviewer": {
        "description": "Expert en securite",
        "system_prompt": "Tu es un expert en securite applicative. Analyse le code pour identifier les vulnerabilites potentielles (injection, XSS, etc.).",
        "model": "sonnet"
    },
    "performance_analyst": {
        "description": "Expert en performance",
        "system_prompt": "Tu es un expert en optimisation de performance. Identifie les goulots d'etranglement et propose des ameliorations.",
        "model": "sonnet"
    },
    "documentation_writer": {
        "description": "Redacteur de documentation",
        "system_prompt": "Tu es un technical writer. Redige une documentation claire et complete.",
        "model": "haiku"
    }
}

print("Agents definis:")
for name, config in custom_agents.items():
    print(f"  - {name}: {config['description']}")

In [ ]:
# Simuler l'agent security_reviewer
agent_config = custom_agents["security_reviewer"]

stdout, stderr, code = run_claude(
    f"""{agent_config['system_prompt']}

Analyse ce code:
{utils_content}

Identifie les problemes de securite potentiels.""",
    model=agent_config['model']
)
print("=== Agent: security_reviewer ===")
print_response(stdout, stderr, code)

In [ ]:
# Simuler l'agent performance_analyst
agent_config = custom_agents["performance_analyst"]

stdout, stderr, code = run_claude(
    f"""{agent_config['system_prompt']}

Analyse ce code:
{utils_content}

Identifie les problemes de performance et propose des optimisations.""",
    model=agent_config['model']
)
print("=== Agent: performance_analyst ===")
print_response(stdout, stderr, code)

## 6. Parallelisation (Concept)

Claude Code peut lancer **jusqu'a 10 agents en parallele** pour accelerer les taches complexes.

En mode interactif, cela se fait automatiquement. En script, on peut simuler avec des threads :

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

def run_agent_task(agent_name, prompt):
    """Execute une tache d'agent et retourne le resultat."""
    start = time.time()
    stdout, stderr, code = run_claude(prompt, model="haiku", timeout=30)
    duration = time.time() - start
    return {
        "agent": agent_name,
        "response": stdout[:500] + "..." if len(stdout) > 500 else stdout,
        "success": code == 0,
        "duration": round(duration, 2)
    }

# Definir les taches
tasks = [
    ("Analyse structure", "Resume la structure de ce code en 3 points"),
    ("Liste fonctions", "Liste toutes les fonctions avec leur signature"),
    ("Identifie types", "Identifie tous les types de donnees utilises")
]

print("Taches definies:")
for name, _ in tasks:
    print(f"  - {name}")

In [ ]:
# Executer en parallele (ATTENTION: consomme des tokens)
# Decommentez pour executer

# with ThreadPoolExecutor(max_workers=3) as executor:
#     futures = {
#         executor.submit(run_agent_task, name, f"{prompt}\n\nCode:\n{utils_content}"): name
#         for name, prompt in tasks
#     }
#     
#     for future in as_completed(futures):
#         result = future.result()
#         print(f"\n=== {result['agent']} ({result['duration']}s) ===")
#         print(result['response'])

print("(Execution parallele desactivee - decommentez pour tester)")

## 7. Exercices

In [ ]:
# EXERCICE 1 : Creez un agent "test_generator" 
# qui genere des tests unitaires pour une fonction donnee

# Votre code ici


In [ ]:
# EXERCICE 2 : Utilisez l'agent Plan pour planifier
# l'ajout d'une fonctionnalite de votre choix

# Votre code ici


## 8. Resume

Dans ce notebook, nous avons appris :

- **Agent Explore** : Recherche rapide en lecture seule
- **Agent Plan** : Planification et decomposition de taches
- **Subagents personnalises** : Definition d'agents specialises
- **Parallelisation** : Executer plusieurs agents simultanement

### Prochaine etape

Dans le notebook final, nous verrons l'**automatisation avancee** : pipelines, hooks, et integration.

-> [05-Claude-CLI-Automatisation.ipynb](05-Claude-CLI-Automatisation.ipynb)